In [6]:
import pandas as pd
df = pd.read_json("/gpuhome/shanto1/Codellama_test/vcp all files/test_python.json", lines=True)
df.head()

PostId                                        code_before  \
0  73672870  You created batches of one element each by set...   
1  73603795  You are doing two different operations in your...   
2  73694160  You need to set ServerName to the hostname in ...   
3  11597911  The main advantages are that it makes multi-li...   
4  73734038  > the first sort (because it's the worst case)...   

                                          code_after           AcceptanceDate  \
0  You created batches of one element each by set...  2022-09-10 00:00:00 UTC   
1  You are doing two different operations in your...  2022-09-05 00:00:00 UTC   
2  You need to set ServerName to the hostname in ...  2022-09-12 00:00:00 UTC   
3  The main advantages are that it makes multi-li...  2012-07-22 00:00:00 UTC   
4  The annoying thing in this exercise is that th...  2022-09-15 00:00:00 UTC   

                  LastEditDate  EditorId    CommentDate  CommenterId  \
0   2022-09-12 11:17:50.63 UTC  14774959  9/10/22 18:36    6859580.0   
1  2022-09-05 01:55:28.487 UTC   5854731    9/5/22 1:51    5854731.0   
2  2022-09-12 19:12:14.757 UTC    374642  9/12/22 20:32   19979668.0   
3   2022-09-02 21:12:10.99 UTC   4518341   9/6/20 12:11    4972447.0   
4   2022-09-15 16:00:41.16 UTC   5459839  9/15/22 15:50   12671057.0   

                                             comment  CommentScore  CTMLabel  \
0  Just so I understand, can you show me how the ...             1         1   
1  Ok, Can you try this `test[["ex1","ex2"]] = te...             1         1   
2  It worked! I changed the second line to "Serve...             1         1   
3                   This answer is really beautiful.             1         1   
4  How do you know that n is len(a)? That's not g...             1         1   

   CotLabel                              SO_Revisions_Link  verdict  \
0         0  https://stackoverflow.com/questions/73672870/        0   
1         0  https://stackoverflow.com/questions/73603795/        1   
2         1  https://stackoverflow.com/questions/73694160/        1   
3         0  https://stackoverflow.com/questions/11597911/        0   
4         0  https://stackoverflow.com/questions/73734038/        0   

  Comment Type Confidence  
0     Question       0.75  
1     Solution       0.75  
2         Flaw        0.5  
3         Flaw        0.5  
4     Question       0.85

In [ ]:
%pip install -q neo4j langchain openai pandas scikit-learn matplotlib seaborn


In [ ]:
%pip install -q sentence-transformers

In [1]:
%pip install -q -U langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q -U langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -q faiss-cpu


In [4]:
%pip install -q -U langchain-community


Note: you may need to restart the kernel to use updated packages.


#RAG (SOUP Data)

In [1]:
# STEP 1: Setup dependencies
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOpenAI
from langchain_community.docstore.document import Document
from sklearn.metrics import classification_report
import openai
import json
import os


/gpuhome/shanto1/miniconda/envs/codellama/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# STEP 2: Load train.json into documents for indexing
with open("/gpuhome/shanto1/Codellama_test/vcp all files/train.json") as f:
    train_data = json.load(f)

documents = []
for item in train_data:
    content = f"Comment: {item['comment']}\nBefore: {item['code_before']}\nAfter: {item['code_after']}\nLabel: {'Valid' if item['valid'] == 1 else 'Invalid'}"
    documents.append(Document(page_content=content))


In [ ]:
import openai
import os
from typing import Optional

# Initialize OpenAI client securely from environment or interactive prompt
api_key = os.getenv("")
if not api_key:
    import getpass
    api_key = getpass.getpass("Enter your OpenAI API key (hidden input): ")

client = openai.OpenAI(api_key=api_key)
print("OpenAI client initialized.")


OpenAI client initialized.


In [ ]:
# STEP 3: Load training data and create documents
with open("") as f:
    train_data = json.load(f)

In [5]:
documents = []
for item in train_data:
    content = f"Comment: {item['comment']}\nBefore: {item['code_before']}\nAfter: {item['code_after']}\nLabel: {'Valid' if item['valid'] == 1 else 'Invalid'}"
    documents.append(Document(page_content=content))


In [ ]:
# STEP 4: Build FAISS vectorstore
embedding = OpenAIEmbeddings()  # Uses OPENAI_API_KEY from environment
vectorstore = FAISS.from_documents(documents, embedding)


In [11]:
# STEP 5: Define retrieval function
retriever = vectorstore.as_retriever(search_type="similarity", k=3)

def get_similar_examples(comment):
    # Use the invoke method (works in LangChain 0.1.0+) 
    # If invoke fails, fall back to direct vectorstore search
    try:
        results = retriever.invoke(comment)
    except AttributeError:
        # Fallback for older LangChain versions
        results = vectorstore.similarity_search(comment, k=3)
    return results

def format_context(docs):
    return "\n\n".join([doc.page_content for doc in docs])


In [8]:
# STEP 6: Define prediction function
def predict(comment, code_before, code_after):
    examples = get_similar_examples(comment)
    context = format_context(examples)

    prompt = f"""
You are a Stack Overflow moderator. You are given a post-edit scenario:
- A comment made on Stack Overflow: {comment}
- The code before edit: {code_before}
- The code after edit: {code_after}

Some previous related comment-edit pairs:
{context}

Decide if the comment justifies the code edit. Answer with "Valid" or "Invalid" only.
""".strip()

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [12]:
# === STEP 7: Evaluate on test.json ===
def load_json_flexible(path):
    # Try to load as a single JSON object/array first, otherwise fall back to JSON Lines (one JSON object per line)
    with open(path, 'r') as f:
        text = f.read().strip()
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        data = []
        with open(path, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError:
                    # skip malformed lines
                    continue
        return data

test_data = load_json_flexible("/gpuhome/shanto1/Codellama_test/vcp all files/test_python.json")

y_true, y_pred = [], []

for item in test_data:
    # use .get to avoid KeyError if a field is missing
    gt = "Valid" if item.get("verdict", 0) == 1 else "Invalid"
    pred = predict(item.get("comment", ""), item.get("code_before", ""), item.get("code_after", ""))

    y_true.append(gt.lower())
    y_pred.append(pred.lower())
    print(f"GT: {gt}, PRED: {pred}")
    print(f"Comment: {item.get('comment', '')}")
    print(f"Code before: {item.get('code_before', '')}")
    print(f"Code after: {item.get('code_after', '')}")
    examples = get_similar_examples(item.get('comment', ''))
    # printing only summaries to avoid overly verbose prints
    print(f"Examples (count={len(examples)}):")
    for ex in examples:
        print("-", ex.page_content[:200].replace("\n", " "), "...")
    print("\n")

print(classification_report(y_true, y_pred))

GT: Invalid, PRED: Valid
Comment: Just so I understand, can you show me how the access with batch_x[0] would look?
Code before: You created batches of one element each by setting `batch_size=1`. 

So if you do:

    data_iterator = data.as_numpy_iterator()
    batch = data_iterator.next()

You are only accessing one image because your batch only has one image in it. To get the next batch, and so, the next image, you can call again `data_iterator.next()`.

However if you only want to iterate through your dataset and print the images, you can do like this:

    data = tf.keras.utils.image_dataset_from_directory('img',batch_size=1,image_size=(171,256))
    for x, y in data:
      print("image: {}, label: {}".format(x,y))


Of course you can also set a higher value of `batch_size` to access more images at a time.


If you want to access an image at a time, you can either:

----------

**Method 1** (advised): iterate through your batches, and through the images within a batch:

    import n

In [13]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

     invalid       0.87      0.72      0.79       429
       valid       0.45      0.68      0.54       143

    accuracy                           0.71       572
   macro avg       0.66      0.70      0.66       572
weighted avg       0.76      0.71      0.73       572



In [14]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

# normalize to lowercase strings to match available data
y_true_norm = [str(v).lower() for v in y_true]
y_pred_norm = [str(v).lower() for v in y_pred]

labels = ['invalid', 'valid']
cm = confusion_matrix(y_true_norm, y_pred_norm, labels=labels)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax, cmap='Blues', values_format='d')
ax.set_title("Confusion Matrix")
plt.tight_layout()
plt.show()

print("Labels order:", labels)
print("Confusion matrix:\n", cm)

Labels order: ['invalid', 'valid']
Confusion matrix:
 [[309 120]
 [ 46  97]]


In [ ]:
import pandas as pd

filename = "/gpuhome/shanto1/Codellama_test/vcp all files/results/rag_responses_saved.csv"

if "rag_results" in globals():
    df_to_save = rag_results.copy()
    # Attach y_pred if available and lengths align
    if "y_pred" in globals() and len(df_to_save) == len(y_pred):
        df_to_save["rag_pred"] = pd.Series(y_pred)
    # Select only the required columns if they exist
    columns_to_save = ["code_before", "code_after", "comment", "gt", "rag_pred"]
    # If "gt" is not present, try "gt_label"
    if "gt" not in df_to_save.columns and "gt_label" in df_to_save.columns:
        df_to_save["gt"] = df_to_save["gt_label"]
    available_columns = [col for col in columns_to_save if col in df_to_save.columns]
    if not available_columns:
        print("No matching columns found to save. Saving all columns instead.")
        df_to_save.to_csv(filename, index=False)
    else:
        df_to_save = df_to_save[available_columns]
        df_to_save.to_csv(filename, index=False)
    print(f"Saved rag_results to {filename} (rows={len(df_to_save)})")
elif "y_pred" in globals() and "y_true" in globals():
    # If rag_results is not available, save basic info
    df_to_save = pd.DataFrame({
        "gt": y_true,
        "rag_pred": y_pred
    })
    df_to_save.to_csv(filename, index=False)
    print(f"Saved predictions to {filename} (rows={len(df_to_save)})")
else:
    print("No rag_results or prediction lists (y_pred/y_true) found in the notebook namespace.")

Saved predictions to /gpuhome/shanto1/Codellama_test/vcp all files/results/rag_responses_saved.csv (rows=572)


In [22]:
test_py= pd.read_json("/gpuhome/shanto1/Codellama_test/vcp all files/test_python.json", lines=True)
test_py.head()

PostId                                        code_before  \
0  73672870  You created batches of one element each by set...   
1  73603795  You are doing two different operations in your...   
2  73694160  You need to set ServerName to the hostname in ...   
3  11597911  The main advantages are that it makes multi-li...   
4  73734038  > the first sort (because it's the worst case)...   

                                          code_after           AcceptanceDate  \
0  You created batches of one element each by set...  2022-09-10 00:00:00 UTC   
1  You are doing two different operations in your...  2022-09-05 00:00:00 UTC   
2  You need to set ServerName to the hostname in ...  2022-09-12 00:00:00 UTC   
3  The main advantages are that it makes multi-li...  2012-07-22 00:00:00 UTC   
4  The annoying thing in this exercise is that th...  2022-09-15 00:00:00 UTC   

                  LastEditDate  EditorId    CommentDate  CommenterId  \
0   2022-09-12 11:17:50.63 UTC  14774959  9/10/22 18:36    6859580.0   
1  2022-09-05 01:55:28.487 UTC   5854731    9/5/22 1:51    5854731.0   
2  2022-09-12 19:12:14.757 UTC    374642  9/12/22 20:32   19979668.0   
3   2022-09-02 21:12:10.99 UTC   4518341   9/6/20 12:11    4972447.0   
4   2022-09-15 16:00:41.16 UTC   5459839  9/15/22 15:50   12671057.0   

                                             comment  CommentScore  CTMLabel  \
0  Just so I understand, can you show me how the ...             1         1   
1  Ok, Can you try this `test[["ex1","ex2"]] = te...             1         1   
2  It worked! I changed the second line to "Serve...             1         1   
3                   This answer is really beautiful.             1         1   
4  How do you know that n is len(a)? That's not g...             1         1   

   CotLabel                              SO_Revisions_Link  verdict  \
0         0  https://stackoverflow.com/questions/73672870/        0   
1         0  https://stackoverflow.com/questions/73603795/        1   
2         1  https://stackoverflow.com/questions/73694160/        1   
3         0  https://stackoverflow.com/questions/11597911/        0   
4         0  https://stackoverflow.com/questions/73734038/        0   

  Comment Type Confidence  
0     Question       0.75  
1     Solution       0.75  
2         Flaw        0.5  
3         Flaw        0.5  
4     Question       0.85

In [23]:
rag=pd.read_csv("/gpuhome/shanto1/Codellama_test/vcp all files/results/rag_responses_saved.csv")
rag.head()

rag_pred       gt
0    valid  invalid
1    valid    valid
2    valid    valid
3    valid  invalid
4  invalid  invalid

In [24]:
rag_with_test = pd.concat([rag.reset_index(drop=True), test_py.reset_index(drop=True)], axis=1)
rag_with_test.head()

rag_pred       gt    PostId  \
0    valid  invalid  73672870   
1    valid    valid  73603795   
2    valid    valid  73694160   
3    valid  invalid  11597911   
4  invalid  invalid  73734038   

                                         code_before  \
0  You created batches of one element each by set...   
1  You are doing two different operations in your...   
2  You need to set ServerName to the hostname in ...   
3  The main advantages are that it makes multi-li...   
4  > the first sort (because it's the worst case)...   

                                          code_after           AcceptanceDate  \
0  You created batches of one element each by set...  2022-09-10 00:00:00 UTC   
1  You are doing two different operations in your...  2022-09-05 00:00:00 UTC   
2  You need to set ServerName to the hostname in ...  2022-09-12 00:00:00 UTC   
3  The main advantages are that it makes multi-li...  2012-07-22 00:00:00 UTC   
4  The annoying thing in this exercise is that th...  2022-09-15 00:00:00 UTC   

                  LastEditDate  EditorId    CommentDate  CommenterId  \
0   2022-09-12 11:17:50.63 UTC  14774959  9/10/22 18:36    6859580.0   
1  2022-09-05 01:55:28.487 UTC   5854731    9/5/22 1:51    5854731.0   
2  2022-09-12 19:12:14.757 UTC    374642  9/12/22 20:32   19979668.0   
3   2022-09-02 21:12:10.99 UTC   4518341   9/6/20 12:11    4972447.0   
4   2022-09-15 16:00:41.16 UTC   5459839  9/15/22 15:50   12671057.0   

                                             comment  CommentScore  CTMLabel  \
0  Just so I understand, can you show me how the ...             1         1   
1  Ok, Can you try this `test[["ex1","ex2"]] = te...             1         1   
2  It worked! I changed the second line to "Serve...             1         1   
3                   This answer is really beautiful.             1         1   
4  How do you know that n is len(a)? That's not g...             1         1   

   CotLabel                              SO_Revisions_Link  verdict  \
0         0  https://stackoverflow.com/questions/73672870/        0   
1         0  https://stackoverflow.com/questions/73603795/        1   
2         1  https://stackoverflow.com/questions/73694160/        1   
3         0  https://stackoverflow.com/questions/11597911/        0   
4         0  https://stackoverflow.com/questions/73734038/        0   

  Comment Type Confidence  
0     Question       0.75  
1     Solution       0.75  
2         Flaw        0.5  
3         Flaw        0.5  
4     Question       0.85

In [25]:
rag_with_test.to_csv("/gpuhome/shanto1/Codellama_test/vcp all files/results/rag_responses formated_python data.csv")

In [27]:
# Filter rag_with_test for rows where the RAG prediction is 'valid' (case-insensitive)
# Handle possible column names used in the notebook ('rag_pred' or 'rag_predict')
candidate_cols = [c for c in ('rag_pred', 'rag_predict') if c in rag_with_test.columns]
if not candidate_cols:
    raise KeyError("No 'rag_pred' or 'rag_predict' column found in rag_with_test.")

col = candidate_cols[0]
rag_with_test_valid = rag_with_test[rag_with_test[col].astype(str).str.lower() == 'valid'].reset_index(drop=True)

print(f"Filtered using column: {col}")
print(f"Total rows in rag_with_test: {len(rag_with_test)}")
print(f"Rows with RAG='valid': {len(rag_with_test_valid)}")
display(rag_with_test_valid.head())

Filtered using column: rag_pred
Total rows in rag_with_test: 572
Rows with RAG='valid': 217


rag_pred       gt    PostId  \
0    valid  invalid  73672870   
1    valid    valid  73603795   
2    valid    valid  73694160   
3    valid  invalid  11597911   
4    valid  invalid  73736507   

                                         code_before  \
0  You created batches of one element each by set...   
1  You are doing two different operations in your...   
2  You need to set ServerName to the hostname in ...   
3  The main advantages are that it makes multi-li...   
4  There is probably a more efficient way to code...   

                                          code_after           AcceptanceDate  \
0  You created batches of one element each by set...  2022-09-10 00:00:00 UTC   
1  You are doing two different operations in your...  2022-09-05 00:00:00 UTC   
2  You need to set ServerName to the hostname in ...  2022-09-12 00:00:00 UTC   
3  The main advantages are that it makes multi-li...  2012-07-22 00:00:00 UTC   
4  There is probably a more efficient way to code...  2022-09-16 00:00:00 UTC   

                  LastEditDate  EditorId    CommentDate  CommenterId  \
0   2022-09-12 11:17:50.63 UTC  14774959  9/10/22 18:36    6859580.0   
1  2022-09-05 01:55:28.487 UTC   5854731    9/5/22 1:51    5854731.0   
2  2022-09-12 19:12:14.757 UTC    374642  9/12/22 20:32   19979668.0   
3   2022-09-02 21:12:10.99 UTC   4518341   9/6/20 12:11    4972447.0   
4   2022-09-16 15:33:31.75 UTC  16201558  9/16/22 15:34   16201558.0   

                                             comment  CommentScore  CTMLabel  \
0  Just so I understand, can you show me how the ...             1         1   
1  Ok, Can you try this `test[["ex1","ex2"]] = te...             1         1   
2  It worked! I changed the second line to "Serve...             1         1   
3                   This answer is really beautiful.             1         1   
4  @ouroboros1 true; I added an edit based on you...             1         1   

   CotLabel                              SO_Revisions_Link  verdict  \
0         0  https://stackoverflow.com/questions/73672870/        0   
1         0  https://stackoverflow.com/questions/73603795/        1   
2         1  https://stackoverflow.com/questions/73694160/        1   
3         0  https://stackoverflow.com/questions/11597911/        0   
4         1  https://stackoverflow.com/questions/73736507/        0   

  Comment Type Confidence  
0     Question       0.75  
1     Solution       0.75  
2         Flaw        0.5  
3         Flaw        0.5  
4     Solution       0.55

In [28]:
rag_with_test_valid.to_csv("/gpuhome/shanto1/Codellama_test/vcp all files/results/rag_pred_valid_python.csv")